In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df=pd.read_csv("../input/car-price-prediction/CarPrice_Assignment.csv")
df

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
print ("Rows     : " , df.shape[0])
print ("Columns  : " , df.shape[1])
print ("\nFeatures : \n", df.columns.tolist())
print ("\nMissing values :  ", df.isnull().sum().values.sum())
print ("\nUnique values :  \n", df.nunique())


In [ ]:
print(df['fueltype'].value_counts())
print(df['fueltype'].value_counts().plot.bar())

In [ ]:
print(df['aspiration'].value_counts())
print(df['aspiration'].value_counts().plot.bar())


In [ ]:
print(df['doornumber'].value_counts())
print(df['doornumber'].value_counts().plot.bar())

In [ ]:
print(df['drivewheel'].value_counts())
print(df['drivewheel'].value_counts().plot.bar())


In [ ]:
print(df['carbody'].value_counts(),
df['carbody'].value_counts().plot.bar())

In [ ]:
print(df['drivewheel'].value_counts(),
df['drivewheel'].value_counts().plot.bar())

In [ ]:
iqr=df.quantile(0.75)-df.quantile(0.25)
iqr

In [ ]:
lw=df.quantile(0.25)-(iqr*1.5)
lw

In [ ]:
uw=df.quantile(0.75)+(iqr*1.5)
uw

In [ ]:
sns.boxplot(x=df['compressionratio'],data=df)

In [ ]:
df['compressionratio_new']=np.where(df['compressionratio']<=lw[0],df['compressionratio'].quantile(0.25),df['compressionratio'])
df['compressionratio_new']=np.where(df['compressionratio']>=uw[10],df['compressionratio'].quantile(0.75),df['compressionratio_new'])

In [ ]:
sns.boxplot(x=df['compressionratio_new'],data=df)

In [ ]:
sns.boxplot(x=df['horsepower'],data=df)

In [ ]:
df['horsepower_new']=np.where(df['horsepower']<=lw[0],df['horsepower'].quantile(0.25),df['horsepower'])
df['horsepower_new']=np.where(df['horsepower']>=uw[7],df['horsepower'].quantile(0.75),df['horsepower_new'])

In [ ]:
sns.boxplot(df['horsepower_new'],data=df)

In [ ]:
sns.boxplot(df['peakrpm'],data=df)

In [ ]:
sns.boxplot(df['citympg'],data=df)

In [ ]:
sns.boxplot(df['highwaympg'],data=df)

In [ ]:
df['highwaympg_new']=np.where(df['highwaympg']<=lw[0],df['highwaympg'].quantile(0.25),df['highwaympg'])
df['highwaympg_new']=np.where(df['highwaympg']>=uw[14],df['highwaympg'].quantile(0.75),df['highwaympg_new'])

In [ ]:
sns.boxplot(df['highwaympg_new'],data=df)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
df_columns=("fueltype","aspiration","doornumber","carbody","drivewheel","enginelocation","enginetype","cylindernumber","fuelsystem")

In [ ]:
for col in df_columns:
    df[col]=le.fit_transform(df[col])

In [ ]:
df

In [ ]:
df=df.drop(['CarName'],axis=1)
df

In [ ]:
x=df[['fueltype','aspiration','doornumber','carbody','drivewheel','enginelocation','wheelbase','carlength','carwidth','carheight','curbweight','enginetype','cylindernumber','enginesize','fuelsystem','boreratio','stroke','citympg','compressionratio_new','horsepower_new','highwaympg_new']].values
x

In [ ]:
y=df[['price']].values
y

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=.8,random_state=0)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression(normalize=True)

In [ ]:
lr.fit(x_train,y_train)

In [ ]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    print(errors)
    print(np.mean(errors/test_labels))
    mape = 100 * np.mean(errors / test_labels)
    print(mape)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    
    return accuracy


In [ ]:
lr_regressor=evaluate(lr,x_test,y_test)

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

def cross_val(model):
    pred = cross_val_score(model, X, y, cv=10)
    return pred.mean()

def print_evaluate(true, predicted):  
    mae = metrics.mean_absolute_error(true, predicted)
    mse = metrics.mean_squared_error(true, predicted)
    rmse = np.sqrt(metrics.mean_squared_error(true, predicted))
    r2_square = metrics.r2_score(true, predicted)
    print('MAE:', mae)
    print('MSE:', mse)
    print('RMSE:', rmse)
    print('R2 Square', r2_square)

In [ ]:
test_pred = lr.predict(x_test)
train_pred =lr.predict(x_train)

print('Test set evaluation:\n_____________________________________')
print_evaluate(y_test, test_pred)
print('Train set evaluation:\n_____________________________________')
print_evaluate(y_train, train_pred)